In [22]:
from os import environ
from dotenv import load_dotenv
import datetime
import requests
import base64
import hmac
import hashlib


# Load environment variables from .env
load_dotenv(".env")

AZURE_STORAGE_ACCOUNT_NAME = environ.get("AZURE_STORAGE_ACCOUNT_NAME")
AZURE_ACCESS_KEY = environ.get("AZURE_ACCESS_KEY")

In [38]:
storage_account_name = AZURE_STORAGE_ACCOUNT_NAME
storage_account_key = AZURE_ACCESS_KEY
api_version = '2018-03-28'
request_time = datetime.datetime.utcnow().strftime('%a, %d %b %Y %H:%M:%S GMT')

string_params = {
    'verb': 'GET',
    'Content-Encoding': '',
    'Content-Language': '',
    'Content-Length': "",
    'Content-MD5': '',
    'Content-Type': '',
    'Date': '',
    'If-Modified-Since': '',
    'If-Match': '',
    'If-None-Match': '',
    'If-Unmodified-Since': '',
    'Range': '',
    'CanonicalizedHeaders': 'x-ms-date:' + request_time + '\nx-ms-version:' + api_version + '\n',
    'CanonicalizedResource': '/' + storage_account_name + '/\ncomp:properties\nrestype:service'
}

string_to_sign = (string_params['verb'] + '\n'
                  + string_params['Content-Encoding'] + '\n'
                  + string_params['Content-Language'] + '\n'
                  + string_params['Content-Length'] + '\n'
                  + string_params['Content-MD5'] + '\n'
                  + string_params['Content-Type'] + '\n'
                  + string_params['Date'] + '\n'
                  + string_params['If-Modified-Since'] + '\n'
                  + string_params['If-Match'] + '\n'
                  + string_params['If-None-Match'] + '\n'
                  + string_params['If-Unmodified-Since'] + '\n'
                  + string_params['Range'] + '\n'
                  + string_params['CanonicalizedHeaders']
                  + string_params['CanonicalizedResource'])

def _sign_string(key, string_to_sign):
    key = base64.b64decode(key.encode('utf-8'))
    string_to_sign = string_to_sign.encode('utf-8')
    signed_hmac_sha256 = hmac.HMAC(key, string_to_sign, hashlib.sha256)
    digest = signed_hmac_sha256.digest()
    encoded_digest = base64.b64encode(digest).decode('utf-8')
    return encoded_digest

# signed_string = base64.b64encode(hmac.new(base64.b64decode(storage_account_key), msg=string_to_sign.encode('utf-8'), digestmod=hashlib.sha256).digest()).decode('utf-8')
# print(signed_string)

signed_string = _sign_string(storage_account_key, string_to_sign=string_to_sign)

headers = {
    'x-ms-date' : request_time,
    'x-ms-version' : api_version,
    'Content-Length': "",
    'Authorization' : ('SharedKey ' + storage_account_name + ':' + signed_string)
}

url = ('https://' + storage_account_name + '.blob.core.windows.net/?resource=account')

response = requests.get(url, headers=headers)
print(response.text)

ConnectionError: HTTPSConnectionPool(host="'b2aiuploadtest.blob.core.windows.net", port=443): Max retries exceeded with url: /?resource=account (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000026F5A28AE80>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))